# The Big Bang Theroy

- Link : https://www.imdb.com/title/tt0898266/?ref_=fn_al_tt_1
- Link : https://www.imdb.com/title/tt0898266/episodes/?ref_=tt_eps_sm
- 시즌별 에피소드 정보 가져와서 정리하기 

- 분석에 필요한 데이터 정리
  - Seasons Page
    1. 제목
    2. 날짜
    3. Rating
  - 방영 정보
    1. 제목
    2. Rating
    3. 장르
    4. 줄거리 요약
    5. 날짜
    6. 영상 길이
    7. Director, Writers, Stars (작가와 배우는 링크를 통해 더 자세한 정보 확인 가능)
    8. 리뷰
    9. 언어
    10. 촬영위치
- 각 회차 마다 Seasons Page의 정보가 모두 포함되어 있음
  - Seasons Page에서 각 회차 링크 뽑은 후,
  - Link에 들어가서 데이터 분석 진행하면 될 듯

In [1]:
# 필요한 모듈 불러오기 

import urllib
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup

def open_html(url):
    ###############################################
    # Referer 및 기타 헤더 설정을 통해 403에러 회피
    headers = {
        "User-Agent": "Mozilla/5.0...",
        "Referer": "http://naver.com/previous-page"
    }
    ###############################################
    req = Request(url, headers=headers) # 헤더 설정
    res = urlopen(req)
    return BeautifulSoup(res, 'html.parser')


In [2]:
# 시즌별 회차 링크 저장
base_url = 'https://www.imdb.com/title/tt0898266/episodes/?ref_=tt_eps_sm'
base_soup = open_html(base_url)

prefix = 'https://www.imdb.com'

season_tab = base_soup.find('div', class_='ipc-tabs ipc-tabs--base ipc-tabs--align-left ipc-tabs--display-chip ipc-tabs--inherit')
season_tab = season_tab.find_all('a', class_="ipc-tab ipc-tab-link ipc-tab--on-base")
seasons_link = [prefix + link['href'] for link in season_tab]
seasons_link = ['https://www.imdb.com/title/tt0898266/episodes/?season=1'] + seasons_link

In [135]:
Big_Bang = {}
for season_cnt, season in enumerate(seasons_link):
    soup = open_html(season)
    links = soup.find_all('a', 'ipc-title-link-wrapper')
    Big_Bang[season_cnt+1] = []
    for cnt, link in enumerate(links):
        season_link = prefix + link['href']
        season_soup = open_html(season_link)
        # 예외처리 (마지막 빈칸이 하나 생성되는 오류를 방지)
        try:
            title = season_soup.find('span', class_='hero__primary-text').string
            rating = season_soup.find('span', class_='sc-eb51e184-1 ljxVSS').string
            genre = [text.string for text in season_soup.find('div',
                    class_='ipc-chip-list__scroller').find_all('a')]
            Plot_summary = season_soup.find('span', 'sc-2d37a7c7-0 ftDfUj').string
            Date, Total_time = [text.string for i, text in enumerate(season_soup.find('ul',
                            'ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt').find_all('li')) if i%2==0] 
            Director = season_soup.find('a', 'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').string
            Writers = [name.string for name in season_soup.find('li',
                'ipc-metadata-list__item ipc-metadata-list-item--link').find('ul').find_all('li')]
            Stars = [name.string for name in season_soup.find_all('a', 'sc-bfec09a1-1 KeEFX')]
            Review_summary = season_soup.find('span','sc-27d2f80b-1 dWDIvF').string
            Review = season_soup.find('div','ipc-html-content-inner-div').get_text()

            Big_Bang[season_cnt+1].append({
                'episod':cnt+1,
                'title':title,
                'rating':rating,
                'genre':genre,
                'plot_summary':Plot_summary,
                'date':Date,
                'running time':Total_time,
                'director':Director,
                'writers':Writers,
                'stars':Stars,
                'review summary':Review_summary,
                'review':Review
            })
        except:
            pass

In [170]:
import pandas as pd
# 시즌 1~13
Seasons = {}
for i in Big_Bang:
    Seasons[i] = pd.DataFrame(Big_Bang[i])
    Seasons[i].set_index(keys='episod', inplace=True)

In [173]:
Seasons[3]

,title,rating,genre,plot_summary,date,running time,director,writers,stars,review summary,review
episod,,,,,,,,,,,
1,The Electric Can Opener Fluctuation,8.3,"[Comedy, Romance]","After returning from the North Pole, a conflic...","Episode aired Sep 21, 2009",21m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Steven Molaro]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",Tricked,Leonard preserves a snowflake in a solution of...
2,The Jiminy Conjecture,7.7,"[Comedy, Romance]",Leonard and Penny's first night together goes ...,"Episode aired Sep 28, 2009",21m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Jim Reynolds]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",The Jiminy Conjecture,"Professor Crawley (Lewis Black) mentions ""havi..."
3,The Gothowitz Deviation,8.1,"[Comedy, Romance]",Sheldon attempts to covertly alter Penny's hab...,"Episode aired Oct 5, 2009",20m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Maria Ferrari]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",The Gothowitz Deviation,"When Sheldon (Jim Parsons) says ""There are mor..."
4,The Pirate Solution,8.0,"[Comedy, Romance]",When Raj has to find a new job or be sent back...,"Episode aired Oct 12, 2009",20m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Steve Holland]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",The Pirate Solution,Raj says his friends may know his cousin Sanja...
5,The Creepy Candy Coating Corollary,8.3,"[Comedy, Romance]",Leonard is forced to ask Penny to set Howard u...,"Episode aired Oct 19, 2009",20m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Lee Aronsohn]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",The Creepy Candy Coating Corollary,"""The Mystic Warlords of Ka'a"" card game was cr..."
6,The Cornhusker Vortex,7.4,"[Comedy, Romance]",Leonard tries to learn about football to fit i...,"Episode aired Nov 2, 2009",20m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, David Goetsch]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",The Continuation Problem,The Battlestar Galactica (2004) Cylon toaster ...
7,The Guitarist Amplification,7.8,"[Comedy, Romance]",Sheldon is caught in the middle when Leonard a...,"Episode aired Nov 9, 2009",20m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Richard Rosenstock]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...","""The Guitarist Amplification"" was another grea...",First mention of Sheldon's father's name: George.
8,The Adhesive Duck Deficiency,8.9,"[Comedy, Romance]",None,"Episode aired Nov 16, 2009",21m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Steven Molaro]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...","""The Adhesive Duck Deficiency"" is another high...","In this episode, Kunal Nayyar displayed his ea..."
9,The Vengeance Formulation,8.3,"[Comedy, Romance]",Sheldon plans revenge after Kripke humiliates ...,"Episode aired Nov 23, 2009",19m,Mark Cendrowski,"[Chuck Lorre, Bill Prady, Richard Rosenstock]","[Johnny Galecki, Jim Parsons, Kaley Cuoco, Sim...",Jokery!,When Kripke pipes helium into Sheldon's office...
